In [1]:
import pandas as pd
#pd.set_option("display.max_rows", None)
import re
import numpy as np
import os
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler 
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [2]:
###importing the data and checking through for anormalies 
data2018 = pd.read_csv('2018b table uncleaned .csv', parse_dates=['date'])
data2018.tail(50)
data2018.isnull().sum()
data2018.shape[0]

209921

In [3]:
##creating a check point and droping the first 4 rows##
data_to_clean = data2018
data_to_clean.columns

Index(['viewid', 'date', 'pagePath', 'hits', 'dimension36'], dtype='object')

creating new column to group new model from old model

In [4]:
##creating function for to find out whether car is new or new or old (Newness Type) in pagePath# ##
data_to_clean["newness_type"]=data_to_clean["pagePath"].str.extract('(new|used)')
data_to_clean['newness_type'].fillna('unknown_type', inplace=True)
data_to_clean['newness_type'].unique() ##check unique values 
data_to_clean['newness_type'].value_counts() ##count unique values 

new             159619
used             40980
unknown_type      9322
Name: newness_type, dtype: int64

In [5]:
##creating function for to find out whether car is new or new or old in Dimension36 (As Newness Type) ##
data_to_clean["newness_type2"]=data_to_clean["dimension36"].str.extract('(new|used)')
data_to_clean['newness_type2'].fillna('unknown_type', inplace=True)
data_to_clean['newness_type2'].unique() ##check unique values 
data_to_clean['newness_type2'].value_counts() ##count unique values 

unknown_type    114351
new              79867
used             15703
Name: newness_type2, dtype: int64

In [6]:
###viewing data without unknown newness type##
data_to_clean[data_to_clean['newness_type']== 'unknown_type'].head(5)

,viewid,date,pagePath,hits,dimension36,newness_type,newness_type2
0,129875374,2018-01-01,/vdp,2,https://statefarm.edmunds.com/details/?vin=1FM...,unknown_type,unknown_type
1,129875374,2018-01-01,/vdp,2,https://statefarm.edmunds.com/details/?vin=2G4...,unknown_type,unknown_type
2,129875374,2018-01-01,/vdp,2,https://statefarm.edmunds.com/details/?vin=KL4...,unknown_type,unknown_type
3,129875374,2018-01-01,/srp,4,https://statefarm.edmunds.com/srp/?make=Toyota...,unknown_type,used
335,129875374,2018-01-01,/mobile_lead_form_thank_you_carcode,2,https://www.edmunds.com/inventory/gp_thank_you...,unknown_type,unknown_type


In [19]:
data_to_clean['newness_type'].unique()
data_to_clean['newness_type'].value_counts()

new             159619
used             40980
unknown_type      9322
Name: newness_type, dtype: int64

In [20]:
check = ["newness_type2","newness_type"]
data_to_clean[check][0:5]
#data_to_clean[data_to_clean["newness_type2"]!= data_to_clean["newness_type"]]

,newness_type2,newness_type
0,unknown_type,unknown_type
1,unknown_type,unknown_type
2,unknown_type,unknown_type
3,used,unknown_type
4,unknown_type,used


In [21]:
data_to_clean[data_to_clean["newness_type2"]!= data_to_clean["newness_type"]][0:5]

,viewid,date,pagePath,hits,dimension36,newness_type,newness_type2
3,129875374,2018-01-01,/srp,4,https://statefarm.edmunds.com/srp/?make=Toyota...,unknown_type,used
4,129875374,2018-01-01,/used_model_car_inventory_vin_detail,4,https://www.edmunds.com/Chevrolet/Tahoe/2015/v...,used,unknown_type
5,129875374,2018-01-01,/used_model_car_inventory_vin_detail,7,https://www.edmunds.com/Ford/F-150/2012/vin/1F...,used,unknown_type
6,129875374,2018-01-01,/new_model_core,2,https://www.edmunds.com/acura/ilx/2017/sedan/#...,new,unknown_type
7,129875374,2018-01-01,/new_model_core,2,https://www.edmunds.com/acura/mdx/2017/#edm-en...,new,unknown_type


In [22]:
### merging btoh newness column to have one column## 
data_to_clean.newness_type[data_to_clean.newness_type=='unknown_type']=data_to_clean.newness_type2


##Drop  Newness_type2##
##clean_1 = data_to_clean[data_to_clean.newness_type!= 'unknown_type']
clean_1 = data_to_clean.drop(['newness_type2'], axis=1)##break point after removing newness_type2 data  or newness or use type

###clean_1.drop(['newness_type2'], axis=1, inplace=True) ##code if if I have to remove the unknown newness type 


clean_1.shape[0]

<ipython-input-22-f23abb027719>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_to_clean.newness_type[data_to_clean.newness_type=='unknown_type']=data_to_clean.newness_type2


209921

In [23]:

data_to_clean['newness_type'].value_counts()

new             165815
used             41772
unknown_type      2334
Name: newness_type, dtype: int64

In [24]:
##data_to_clean[data_to_clean['newness_type']== 'unknown_type'].tail(100)

In [25]:
clean_1.head() ##previewing data 


,viewid,date,pagePath,hits,dimension36,newness_type
0,129875374,2018-01-01,/vdp,2,https://statefarm.edmunds.com/details/?vin=1FM...,unknown_type
1,129875374,2018-01-01,/vdp,2,https://statefarm.edmunds.com/details/?vin=2G4...,unknown_type
2,129875374,2018-01-01,/vdp,2,https://statefarm.edmunds.com/details/?vin=KL4...,unknown_type
3,129875374,2018-01-01,/srp,4,https://statefarm.edmunds.com/srp/?make=Toyota...,used
4,129875374,2018-01-01,/used_model_car_inventory_vin_detail,4,https://www.edmunds.com/Chevrolet/Tahoe/2015/v...,used


In [26]:
#####working on the url (dimension36) too extract importanst data#
###first removing the url beginning ###
##removing the beginning url for consistencies in the column 


In [27]:
clean_2= clean_1.copy()
clean_2[0:5]

,viewid,date,pagePath,hits,dimension36,newness_type
0,129875374,2018-01-01,/vdp,2,https://statefarm.edmunds.com/details/?vin=1FM...,unknown_type
1,129875374,2018-01-01,/vdp,2,https://statefarm.edmunds.com/details/?vin=2G4...,unknown_type
2,129875374,2018-01-01,/vdp,2,https://statefarm.edmunds.com/details/?vin=KL4...,unknown_type
3,129875374,2018-01-01,/srp,4,https://statefarm.edmunds.com/srp/?make=Toyota...,used
4,129875374,2018-01-01,/used_model_car_inventory_vin_detail,4,https://www.edmunds.com/Chevrolet/Tahoe/2015/v...,used


In [28]:
####Extracting Values for make, model and year

###first create another variable "url" by modifying "variable dimension36" which will have a spacebar the end for uniformity ##
clean_2['url'] = clean_2['dimension36'].astype(str) + ' '

####Extracting Values for make, model and year
clean_2['Maker']=clean_2['dimension36'].str.extract('(make=[\w+]+\W)')
clean_2['Makerb']=clean_2['url'].str.extract('(make=[a-zA-Z0-9_\%\W]+ $)')
clean_2['Model']=clean_2['dimension36'].str.extract('(model=[\w+-]+\W)')
clean_2['Modelb']=clean_2['url'].str.extract('(model=[a-zA-Z0-9_\%\W]+ $)')
clean_2['Year']=clean_2['dimension36'].str.extract('(year=......)')
clean_2['Extcolor']=clean_2['url'].str.extract('(extcolor=[\w+]+\W)')
clean_2['Intcolor']=clean_2['url'].str.extract('(intcolor=[\w+]+\W)')
####clean_2['ModelB']=clean_2['url'].str.extract('(model=[a-zA-Z0-9_\-]+ $)')


##replace the separator % with & for uniformity##
clean_2['Maker']=clean_2['Maker'].str.replace('%','&')
clean_2['Model']=clean_2['Model'].str.replace('%','&')
clean_2['Year']=clean_2['Year'].str.replace('%','&')
#clean_2['Modelb']=clean_2['Modelb'].str.replace(' ','&')###not necessary because we are splitting on spacebar
#clean_2['Makerb']=clean_2['Makerb'].str.replace(' ','&')###not necessary because we are splitting on spacebar



##Fill all Null values within columns make, model and year####
clean_2['Maker'].fillna('unknown', inplace=True)
clean_2['Makerb'].fillna('unknown', inplace=True)
clean_2['Model'].fillna('unknown', inplace=True)
clean_2['Modelb'].fillna('unknown', inplace=True)
clean_2['Year'].fillna('unknown', inplace=True)
clean_2['Extcolor'].fillna('unknown', inplace=True)
clean_2['Intcolor'].fillna('unknown', inplace=True)


###creating a cleaned column for variable make, model and yea## 
clean_2['OEM']=clean_2['Maker'].apply(lambda st: st[st.find("make=")+5:st.find("\W")])
clean_2['OEMB']=clean_2['Makerb'].apply(lambda st: st[st.find("make=")+5:st.find(" ")])
clean_2['MODEL']=clean_2['Model'].apply(lambda st: st[st.find("model=")+6:st.find("\W")])
clean_2['MODELB']=clean_2['Modelb'].apply(lambda st: st[st.find("model=")+6:st.find(" ")])
clean_2['YEAR_BUILT']=clean_2['Year'].apply(lambda st: st[st.find("year=")+5:st.find("\W")-1])
clean_2['INTCOLOR']=clean_2['Extcolor'].apply(lambda st: st[st.find("extcolor=")+9:st.find("\W")])  ######this will dates without ranges 
clean_2['EXTCOLOR']=clean_2['Intcolor'].apply(lambda st: st[st.find("intcolor=")+9:st.find("\W")]) ###this will get data for dates that have ranges ranges



##replace ow/w values within columns for clarification####
clean_2['OEM'].replace({"ow": "unknown"}, inplace=True)
clean_2['OEMB'].replace({"ow": "unknown"}, inplace=True)
clean_2['MODEL'].replace({"w": "unknown"}, inplace=True)
clean_2['MODELB'].replace({"w": "unknown"}, inplace=True)
clean_2['YEAR_BUILT'].replace({"o": "unknown"}, inplace=True)

clean_2['INTCOLOR'] = clean_2.INTCOLOR.replace(r'^\s*$', np.NaN, regex=True)
clean_2['EXTCOLOR'] = clean_2.INTCOLOR.replace(r'^\s*$', np.NaN, regex=True)
#clean_2['INTCOLOR'].replace({" ": "unknown"}, inplace=True)
clean_2.INTCOLOR.fillna("unknown", inplace=True)
clean_2['EXTCOLOR'].fillna("unknown", inplace=True)




###Merging both OEM and MODEL
clean_2.OEM[clean_2.OEM=='unknown']=clean_2.OEMB
clean_2.MODEL[clean_2.MODEL=='unknown']=clean_2.MODELB


##Drop unwanted columns##
####clean_2.drop(['Maker','Makerb','Model','Modelb','Year','MODELB','pagePath','url', 'OEMB'], axis=1, inplace=True)


###Needed variables 
var= ['viewid', 'date', 'hits','dimension36','newness_type','OEM','MODEL','YEAR_BUILT']
color = ['EXTCOLOR','INTCOLOR']
Data_cleaned_part1 = clean_2[var]
color_df = clean_2[color]


<ipython-input-28-328955843060>:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_2.OEM[clean_2.OEM=='unknown']=clean_2.OEMB
<ipython-input-28-328955843060>:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_2.MODEL[clean_2.MODEL=='unknown']=clean_2.MODELB


In [29]:
#Data_cleaned_part1[8535:8550]

In [30]:
#clean_2['Modelb'].isnull().sum()
#clean_2[clean_2['Year']=='unknown']


In [31]:
clean_2['INTCOLOR'].value_counts()

unknown     197412
Black          932
Crystal        537
White          432
Silver         304
             ...  
Aruba            1
Emerald          1
Woodland         1
Juniper          1
Pace             1
Name: INTCOLOR, Length: 750, dtype: int64

In [33]:
###checking if theres  is color info###
clean_2[clean_2['INTCOLOR']!=''][0:5]
#clean_2[8535:8550]

,viewid,date,pagePath,hits,dimension36,newness_type,url,Maker,Makerb,Model,...,Year,Extcolor,Intcolor,OEM,OEMB,MODEL,MODELB,YEAR_BUILT,INTCOLOR,EXTCOLOR
0,129875374,2018-01-01,/vdp,2,https://statefarm.edmunds.com/details/?vin=1FM...,unknown_type,https://statefarm.edmunds.com/details/?vin=1FM...,unknown,unknown,unknown,...,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown
1,129875374,2018-01-01,/vdp,2,https://statefarm.edmunds.com/details/?vin=2G4...,unknown_type,https://statefarm.edmunds.com/details/?vin=2G4...,unknown,unknown,unknown,...,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown
2,129875374,2018-01-01,/vdp,2,https://statefarm.edmunds.com/details/?vin=KL4...,unknown_type,https://statefarm.edmunds.com/details/?vin=KL4...,unknown,unknown,unknown,...,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown
3,129875374,2018-01-01,/srp,4,https://statefarm.edmunds.com/srp/?make=Toyota...,used,https://statefarm.edmunds.com/srp/?make=Toyota...,make=Toyota&,make=Toyota%26model=Sequoia%26type=used,model=Sequoia&,...,unknown,unknown,unknown,Toyota,Toyota%26model=Sequoia%26type=used,Sequoia,Sequoia%26type=used,unknown,unknown,unknown
4,129875374,2018-01-01,/used_model_car_inventory_vin_detail,4,https://www.edmunds.com/Chevrolet/Tahoe/2015/v...,used,https://www.edmunds.com/Chevrolet/Tahoe/2015/v...,unknown,unknown,unknown,...,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown


EXPORT POINT TO VIEW OUTPUT 

In [34]:
##checking for the correctness of output among variables with values 
#Data_cleaned_part1.to_csv('Data_cleaned_part12018.csv', encoding='utf-8')

CLEANING THE SECOND SEGMENT USING SPLIT

In [35]:


#dataA = Data_cleaned2019_part1[(Data_cleaned2019_part1['YEAR_BUILT']!='unknown') & (Data_cleaned2019_part1['OEM']!='unknown') ]
#dataB = Data_cleaned2019_part1[(Data_cleaned2019_part1['YEAR_BUILT']=='unknown') & (Data_cleaned2019_part1['OEM']=='unknown')]

dataA = Data_cleaned_part1[Data_cleaned_part1['OEM']!='unknown'] 
dataB = Data_cleaned_part1[Data_cleaned_part1['OEM']=='unknown']

In [36]:
dataB

,viewid,date,hits,dimension36,newness_type,OEM,MODEL,YEAR_BUILT
0,129875374,2018-01-01,2,https://statefarm.edmunds.com/details/?vin=1FM...,unknown_type,unknown,unknown,unknown
1,129875374,2018-01-01,2,https://statefarm.edmunds.com/details/?vin=2G4...,unknown_type,unknown,unknown,unknown
2,129875374,2018-01-01,2,https://statefarm.edmunds.com/details/?vin=KL4...,unknown_type,unknown,unknown,unknown
4,129875374,2018-01-01,4,https://www.edmunds.com/Chevrolet/Tahoe/2015/v...,used,unknown,unknown,unknown
5,129875374,2018-01-01,7,https://www.edmunds.com/Ford/F-150/2012/vin/1F...,used,unknown,unknown,unknown
...,...,...,...,...,...,...,...,...
209916,129875374,2018-08-09,2,https://www.edmunds.com/volvo/xc40/2019/thanky...,new,unknown,unknown,unknown
209917,129875374,2018-08-09,4,https://www.edmunds.com/volvo/xc60/2017/vin/YV...,used,unknown,unknown,unknown
209918,129875374,2018-08-09,2,https://www.edmunds.com/volvo/xc60/2017/vin/YV...,used,unknown,unknown,unknown
209919,129875374,2018-08-09,2,https://www.edmunds.com/volvo/xc60/2018/?utm_m...,new,unknown,unknown,unknown


In [38]:
##Creating checkpoint##
###CHeckpoint###
#dataA[1510:1531]
#dataA.to_csv('dataA2020vv.csv',encoding='utf-8')
#dataB.to_csv('dataB2020cc.csv',encoding='utf-8')

var = ['viewid','date','hits','dimension36','newness_type'] 
to_clean = dataB[var]
#to_clean


In [39]:
###first removing the beginning url string if any## 
to_clean["dimension36"].replace(regex=['(http+[\W+\w+]+.edmunds.com)'], value=[''], inplace = True) ##removing the beginning url for consistencies in the column 
to_clean["dimension36"].replace(regex=['(/+used+-)'], value=['/'], inplace = True) ##removing the beginning url for consistencies in the column 
#to_clean["dimension36"].replace(regex=['(\A/+0-9+/)'], value=['/'], inplace = True) ##removing the beginning url for consistencies in the column 
to_clean["dimension36"].replace(regex=['-'], value=['/'], inplace = True) ##removing the beginning url for consistencies in the column 


##splitting the url by (/)
url = to_clean["dimension36"].str.split('/', expand=True)

##naming the split columns ##
url.rename(columns={0:"empty", 1:"OEM", 2:"MODEL", 3:"YEAR_BUILT", 4:"design", 5:"exitpage1", 6:"exitpage2"}, inplace=True)

##dropping unwanted columns 
neededcolumn = ['OEM','MODEL','YEAR_BUILT'] 
useful = url[neededcolumn]

/Users/mayorbaba/opt/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:4509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [42]:
#to_clean[57990:58040]

In [43]:
useful[57990:58040]

,OEM,MODEL,YEAR_BUILT
76326,volkswagen,passat,2018
76327,volkswagen,tiguan,limited
76328,volkswagen,tiguan,2018
76329,volkswagen,tiguan,2018
76330,volkswagen,tiguan,2018
76331,volkswagen,tiguan,2018
76332,volvo,s90,2017
76333,volvo,xc90,2018
76334,acura,ilx,2017
76335,acura,ilx,2017


In [44]:
useful

,OEM,MODEL,YEAR_BUILT
0,details,?vin=1FMCU03Z97KA38428,None
1,details,?vin=2G4WB55K2Y1295996,None
2,details,?vin=KL4CJCSB1DB121425,None
4,Chevrolet,Tahoe,2015
5,Ford,F,150
...,...,...,...
209916,volvo,xc40,2019
209917,volvo,xc60,2017
209918,volvo,xc60,2017
209919,volvo,xc60,2018


In [45]:
##Joining the dataframe back to ##
cleaned = pd.concat([to_clean, useful], axis=1)

In [46]:

cleaned 

,viewid,date,hits,dimension36,newness_type,OEM,MODEL,YEAR_BUILT
0,129875374,2018-01-01,2,/details/?vin=1FMCU03Z97KA38428,unknown_type,details,?vin=1FMCU03Z97KA38428,None
1,129875374,2018-01-01,2,/details/?vin=2G4WB55K2Y1295996,unknown_type,details,?vin=2G4WB55K2Y1295996,None
2,129875374,2018-01-01,2,/details/?vin=KL4CJCSB1DB121425,unknown_type,details,?vin=KL4CJCSB1DB121425,None
4,129875374,2018-01-01,4,/Chevrolet/Tahoe/2015/vin/1GNSKBKC5FR180478/?r...,used,Chevrolet,Tahoe,2015
5,129875374,2018-01-01,7,/Ford/F/150/2012/vin/1FTFW1ET5CFB39172/?utm_so...,used,Ford,F,150
...,...,...,...,...,...,...,...,...
209916,129875374,2018-08-09,2,/volvo/xc40/2019/thankyou.html?submit=true,new,volvo,xc40,2019
209917,129875374,2018-08-09,4,/volvo/xc60/2017/vin/YV449MRR7H2064128/,used,volvo,xc60,2017
209918,129875374,2018-08-09,2,/volvo/xc60/2017/vin/YV449MRRXH2223420/,used,volvo,xc60,2017
209919,129875374,2018-08-09,2,/volvo/xc60/2018/?utm_medium=sem&utm_source=go...,new,volvo,xc60,2018


In [47]:
###checkpoint to view output in excel##
#cleaned.to_csv('cleaned.csv',encoding='utf-8')

In [48]:
##Fill all Null values within columns make, model and year####
cleaned['OEM'].fillna('unknown', inplace=True)
cleaned['MODEL'].fillna('unknown', inplace=True)
cleaned['YEAR_BUILT'].fillna('unknown', inplace=True)

In [49]:
####Next is joining the data together as one ##
joined_clean = cleaned.append(dataA)

####Next is joining the data together as one ##
joined_clean= cleaned.append(dataA)


##joining the color dataframe from the original dataframe
data_with_color = pd.concat([joined_clean, color_df], axis=1)

##clean color again##
data_with_color['INTCOLOR'] = data_with_color.INTCOLOR.replace(r'^\s*$', np.NaN, regex=True)
data_with_color['EXTCOLOR'] = data_with_color.INTCOLOR.replace(r'^\s*$', np.NaN, regex=True)
data_with_color.INTCOLOR.fillna("unknown", inplace=True)
data_with_color['EXTCOLOR'].fillna("unknown", inplace=True)
data_with_color.shape[0]

209921

In [50]:
##checkpoint to check joined data###
#join_plus_color.to_csv('joined_cleaned2018.csv',encoding='utf-8')
data_with_color.shape[0]

209921

In [51]:
###FINAL CLEANING OF MODEL ALL COLUMNS 
###cleaning up the year column to have only numbers###
data_with_color['YEAR_BUILT'] = data_with_color['YEAR_BUILT'].str.replace(r'\D+', '')
data_with_color['YEAR_BUILT'] .replace({'': "unknown"}, inplace=True)
data_with_color['YEAR_BUILT'].value_counts()

###SELECTING THE FINAL DATA
final_2018 = data_with_color.drop(['viewid','dimension36'],  axis=1)
complete = final_2018[(final_2018['YEAR_BUILT']>'1800') & (final_2018['YEAR_BUILT']<'2022')]
complete.shape[0]

<ipython-input-51-46599b86de79>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  data_with_color['YEAR_BUILT'] = data_with_color['YEAR_BUILT'].str.replace(r'\D+', '')


172812

By Checking on the error files that had no date, we can see that they are really not relevant and they only carry information of data such as:
1. lead form certificate 
2. Exiting Thank you page
3. Promotional pages (e.g hyundai test drive )
4. Dealer locator link 
all of the link do not contain data that are relevant for analysis and therefore they will be dropped .


In [52]:
#check point to check through in excel##
#final_2018 
complete.to_csv('18timi_cleaned.csv',encoding='utf-8')

In [53]:
#final_2018[57990:58040]

In [54]:
data_with_color.to_csv('2018timi_cleaned.csv',encoding='utf-8')